In [9]:
ev = evaluation.Evaluation(ml_small)

In [6]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
from models.ConstraintAutoRec import ConstraintAutoRec 
import models.NeuralLogicRec
import tensorflow as tf

importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)
import itertools

ml_small = utils.common.ml_small



In [19]:
nlr = models.NeuralLogicRec.NLR(ml_small['user'], ml_small['dimensions'], epochs=1, embedding_dim=16, batch_size=32)

In [20]:
nlr.epochs = 3
for i in range(5):
    nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])
    print(ev.evaluate_single_thread(nlr))

Epoch #0 Loss at step 19: 0.3947, time: 3.449
Epoch #1 Loss at step 19: 0.3762, time: 3.256
Epoch #2 Loss at step 19: 0.3349, time: 3.261
Batch nr 1 predicted
Batch nr 2 predicted
{'accuracy': 0.23436914355322339, 'precision': 0.0, 'recall': 0.0, 'map@1': 0.0, 'map@5': 0.11666666666666665, 'map@10': 0.058415798611111104, 'diversity@5': 0.3572538489829681, 'diversity@10': 0.37283185916797323, 'epc@5': 0.8794947147621348, 'epc@10': 0.8977243499191544, 'epd@5': 0.36062689803334536, 'name': 'NeuralLogicRec', 'latent_dim': 128, 'epochs': 3, 'batch_size': 32}
Epoch #0 Loss at step 19: 0.1828, time: 3.268
Epoch #1 Loss at step 19: 0.1711, time: 3.283
Epoch #2 Loss at step 19: 0.1588, time: 3.257
Batch nr 1 predicted
Batch nr 2 predicted
{'accuracy': 0.4002061469265367, 'precision': 0.8214893617021277, 'recall': 0.34585205196697427, 'map@1': 0.5, 'map@5': 0.24192708333333332, 'map@10': 0.14330667162698413, 'diversity@5': 0.31756096761450514, 'diversity@10': 0.3346572183511819, 'epc@5': 0.79270

In [13]:
ev.evaluate_single_thread(nlr)

Batch nr 1 predicted
Batch nr 2 predicted


{'accuracy': 0.37714345952023987,
 'precision': 0.8242212848799482,
 'recall': 0.305622166909503,
 'map@1': 0.5,
 'map@5': 0.3,
 'map@10': 0.2411111111111111,
 'diversity@5': 0.3408347497998069,
 'diversity@10': 0.32814213143828574,
 'epc@5': 0.7830880489966372,
 'epc@10': 0.8210207698993306,
 'epd@5': 0.36000971010197397,
 'name': 'NeuralLogicRec',
 'latent_dim': 128,
 'epochs': 5,
 'batch_size': 32}

In [6]:
# print(np.flip(np.argsort(nlr.predict_single_user(3))))
pred = nlr.model([4])
# inf = models.NeuralLogicRec.map_inference(nlr.model, pred)
print(np.sort(pred['likes']))
# print(np.sort(inf['likes'].numpy()))
# nlr.model(np.array(1), np.array(1))
# ev.evaluate(nlr)
# pred = nlr.model(tf.convert_to_tensor([1, 2, 3], dtype=tf.int32))

[[3.0696392e-06 3.9041042e-06 5.6028366e-06 ... 9.8202431e-01
  9.8493266e-01 9.8527825e-01]]


In [7]:
u = tf.convert_to_tensor([x for x in itertools.permutations(range(600), 2)])
u1 = u[:,0]
u2 = u[:,1]

In [86]:
num_movies = 10
num_user = 3
rated = tf.convert_to_tensor(np.arange(num_movies * num_user).reshape([num_user, num_movies]) % 3 == 0)
seen = tf.convert_to_tensor(np.arange(num_movies * num_user, num_movies * num_user * 2).reshape([num_user, num_movies]))
users = tf.convert_to_tensor(np.arange(num_user))
users = tf.convert_to_tensor([20, 10, 5, 0, 40])
print(rated)
print(seen)

tf.Tensor(
[[ True False False  True False False  True False False  True]
 [False False  True False False  True False False  True False]
 [False  True False False  True False False  True False False]], shape=(3, 10), dtype=bool)
tf.Tensor(
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]], shape=(3, 10), dtype=int64)


In [87]:
user_cross = tf.convert_to_tensor([x for x in itertools.permutations(users.numpy(), 2)])
print(user_cross)
rated_cross = tf.convert_to_tensor([tf.stack([rated[x[0]], rated[x[1]]], axis=0) for x in itertools.permutations(range(len(users.numpy())), 2)])
# print(rated_cross)

tf.Tensor(
[[20 10]
 [20  5]
 [20  0]
 [20 40]
 [10 20]
 [10  5]
 [10  0]
 [10 40]
 [ 5 20]
 [ 5 10]
 [ 5  0]
 [ 5 40]
 [ 0 20]
 [ 0 10]
 [ 0  5]
 [ 0 40]
 [40 20]
 [40 10]
 [40  5]
 [40  0]], shape=(20, 2), dtype=int32)


InvalidArgumentError: slice index 3 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [88]:
len(user_cross)

20

In [89]:
user_emb = tf.expand_dims(tf.nn.embedding_lookup(nlr.model.user_embedding, user_cross), axis=2)
print(user_emb[:, 0])
user_embedd = tf.tile(user_emb, [1, 1, num_movies, 1])
expanded_embedd = tf.reshape(nlr.model.item_embedding, [1, 1, num_movies, 24])
item_embedd = tf.tile(expanded_embedd, [len(user_cross), 2, 1, 1])
print(item_embed.shape)
print(user_embedd.shape)
inp = tf.concat([user_embedd, item_embedd], axis=-1)

tf.Tensor(
[[[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   0.08901728
   -0.90769035 -1.6418009  -0.3131709 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   

InvalidArgumentError: Input to reshape is a tensor with 83048 values, but the requested shape has 240 [Op:Reshape]

In [90]:
result = tf.squeeze(nlr.model.likes_estimator(inp))

NameError: name 'inp' is not defined

In [84]:
num_movies = 5
users = tf.convert_to_tensor([1, 10, 100])
embed_user = tf.expand_dims(users, axis=1)
users_1 = tf.tile(tf.expand_dims(embed_user, axis=1), [1, len(users), 1])
users_2 = tf.tile(tf.expand_dims(embed_user, axis=0), [len(users),1, 1])
# print(embed_user)
# embed_user = tf.tile(embed_user, [1, num_movies, 1])
# expanded_embedd = tf.expand_dims(nlr.model.item_embedding[0:num_movies], axis=0)
# embed_item = tf.tile(expanded_embedd, [len(users), 1, 1])
sim_input = tf.reduce_sum(tf.concat([users_1, users_2], axis=-1), axis=-1)
print(sim_input)

tf.Tensor(
[[  2  11 101]
 [ 11  20 110]
 [101 110 200]], shape=(3, 3), dtype=int32)


In [99]:
multi_ratings = tf.tile(tf.expand_dims(rated, axis=0), [3, 1, 1])

In [13]:
a = tf.convert_to_tensor(10)
# b = tf.expand_dims(tf.convert_to_tensor(20), axis=0)
# c = tf.convert_to_tensor([1,2,3,4])
len(a.shape) == 0

True

In [18]:
tf.concat([a,b, c], axis=0)

<tf.Tensor: id=134207, shape=(6,), dtype=int32, numpy=array([10, 20,  1,  2,  3,  4], dtype=int32)>